In [1]:
!pip install geopandas


In [2]:
import altair as alt
import geopandas as gpd
import pandas as pd
import json

# Charger les données
names = pd.read_csv("dpt2020.csv", sep=";")
names = names[(names.preusuel != '_PRENOMS_RARES') & (names.dpt != 'XX')]
names['annais'] = names['annais'].astype(int)
names['dpt'] = names['dpt'].astype(str)

# Charger départements
depts = gpd.read_file('departements-version-simplifiee.geojson')
depts['code'] = depts['code'].astype(str)
depts = depts[['code', 'nom', 'geometry']]

just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')
names.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
2996411,54,Meurthe-et-Moselle,"POLYGON ((5.47091 49.49721, 5.54118 49.51526, ...",2,MARIANNE,1981,54,8
156115,17,Charente-Maritime,"MULTIPOLYGON (((-1.4809 46.21003, -1.4528 46.2...",1,ARNAUD,2004,17,6
2176552,71,Saône-et-Loire,"POLYGON ((4.11597 47.12334, 4.15377 47.11456, ...",2,COLINE,1994,71,7
172074,84,Vaucluse,"MULTIPOLYGON (((4.89291 44.36482, 4.90663 44.3...",1,AUGUSTIN,1924,84,5
3236253,59,Nord,"MULTIPOLYGON (((3.0404 50.15971, 3.06301 50.17...",2,NAÏMA,1988,59,4


In [3]:
from IPython.display import display
pd.set_option('display.max_colwidth', None)


# Étape 1 : regrouper par année, sexe, prénom et sommer les naissances
top_name_overall = (
    just_names
    .groupby(['annais', 'sexe', 'preusuel'], as_index=False)['nombre']
    .sum()
)

# Étape 2 : trier et garder le top 5 par année et sexe
top_5 = (
    top_name_overall
    .sort_values(['annais', 'sexe', 'nombre'], ascending=[True, True, False])
    .groupby(['annais', 'sexe'])
    .head(5)
)

# Étape 3 : créer une colonne "Top_5_name" combinée (format texte)
top_5['name_count'] = top_5['preusuel'] + ' ' + top_5['nombre'].astype(str)

# Étape 4 : regrouper en liste par (annais, sexe)
top_name_overall = (
    top_5.groupby(['annais', 'sexe'])['name_count']
         .apply(list)
         .reset_index()
         .rename(columns={'name_count': 'Top_5_name_year'})
)

display(top_name_overall.head(10))

,annais,sexe,Top_5_name_year
0,1900,1,"[JEAN 14097, LOUIS 9052, PIERRE 7454, JOSEPH 7259, HENRI 6919]"
1,1900,2,"[MARIE 48713, JEANNE 13981, MARGUERITE 8058, GERMAINE 6980, LOUISE 6696]"
2,1901,1,"[JEAN 15634, LOUIS 10061, PIERRE 8450, JOSEPH 7891, MARCEL 7582]"
3,1901,2,"[MARIE 52150, JEANNE 14963, MARGUERITE 9009, GERMAINE 7688, LOUISE 7199]"
4,1902,1,"[JEAN 16364, LOUIS 10436, PIERRE 8978, MARCEL 8187, HENRI 8026]"
5,1902,2,"[MARIE 51857, JEANNE 14927, MARGUERITE 9101, GERMAINE 8335, LOUISE 6981]"
6,1903,1,"[JEAN 16535, LOUIS 9988, PIERRE 9074, MARCEL 8600, HENRI 8139]"
7,1903,2,"[MARIE 50424, JEANNE 15017, MARGUERITE 8846, GERMAINE 7934, LOUISE 6866]"
8,1904,1,"[JEAN 16944, LOUIS 10415, PIERRE 9215, MARCEL 8874, JOSEPH 8114]"
9,1904,2,"[MARIE 50131, JEANNE 14909, MARGUERITE 9003, GERMAINE 8260, LOUISE 6841]"


In [4]:
# 1. S'assurer qu'on a un DataFrame de base
top_name_overall = top_name_overall.copy()
max_year = just_names['annais'].max()

# 2. Fonction pour calculer le top 5 d'une fenêtre glissante


def get_top_5_decade(year, sexe):
    end_year = min(year + 10, max_year)
    subset = just_names[(just_names['annais'] >= year) &
                        (just_names['annais'] <= end_year) &
                        (just_names['sexe'] == sexe)]

    if subset.empty:
        return []

    grouped = (
        subset.groupby('preusuel')['nombre']
        .sum()
        .sort_values(ascending=False)
        .head(5)
    )
    return [f"{name} {count}" for name, count in grouped.items()]


# 3. Appliquer la fonction ligne par ligne
top_name_overall['top_5_name_decade'] = top_name_overall.apply(
    lambda row: get_top_5_decade(row['annais'], row['sexe']),
    axis=1
)

top_name_overall.head(10)

,annais,sexe,Top_5_name_year,top_5_name_decade
0,1900,1,"[JEAN 14097, LOUIS 9052, PIERRE 7454, JOSEPH 7259, HENRI 6919]","[JEAN 195366, LOUIS 115335, PIERRE 105357, MARCEL 101632, ANDRÉ 93708]"
1,1900,2,"[MARIE 48713, JEANNE 13981, MARGUERITE 8058, GERMAINE 6980, LOUISE 6696]","[MARIE 535905, JEANNE 166163, MARGUERITE 98455, GERMAINE 86971, LOUISE 73834]"
2,1901,1,"[JEAN 15634, LOUIS 10061, PIERRE 8450, JOSEPH 7891, MARCEL 7582]","[JEAN 201319, LOUIS 116793, PIERRE 108889, MARCEL 105839, ANDRÉ 100144]"
3,1901,2,"[MARIE 52150, JEANNE 14963, MARGUERITE 9009, GERMAINE 7688, LOUISE 7199]","[MARIE 529485, JEANNE 166939, MARGUERITE 98801, GERMAINE 87073, MADELEINE 75916]"
4,1902,1,"[JEAN 16364, LOUIS 10436, PIERRE 8978, MARCEL 8187, HENRI 8026]","[JEAN 207337, LOUIS 117876, PIERRE 112350, MARCEL 109617, ANDRÉ 106761]"
5,1902,2,"[MARIE 51857, JEANNE 14927, MARGUERITE 9101, GERMAINE 8335, LOUISE 6981]","[MARIE 521030, JEANNE 167495, MARGUERITE 98148, GERMAINE 86802, MADELEINE 78518]"
6,1903,1,"[JEAN 16535, LOUIS 9988, PIERRE 9074, MARCEL 8600, HENRI 8139]","[JEAN 212871, LOUIS 118321, PIERRE 115580, ANDRÉ 113248, MARCEL 113070]"
7,1903,2,"[MARIE 50424, JEANNE 15017, MARGUERITE 8846, GERMAINE 7934, LOUISE 6866]","[MARIE 511869, JEANNE 167915, MARGUERITE 97332, GERMAINE 85582, MADELEINE 80830]"
8,1904,1,"[JEAN 16944, LOUIS 10415, PIERRE 9215, MARCEL 8874, JOSEPH 8114]","[JEAN 218424, LOUIS 118907, PIERRE 118801, ANDRÉ 118320, MARCEL 115283]"
9,1904,2,"[MARIE 50131, JEANNE 14909, MARGUERITE 9003, GERMAINE 8260, LOUISE 6841]","[MARIE 503753, JEANNE 168266, MARGUERITE 96253, GERMAINE 84242, MADELEINE 83070]"


In [5]:
import pandas as pd
import altair as alt

# 🔁 Fonction de transformation


def prepare_barchart_data(df, sexe, colname):
    subset = df[df['sexe'] == sexe][['annais', colname]].copy()
    subset = subset.explode(colname)
    subset[['preusuel', 'nombre']] = subset[colname].str.extract(
        r'([A-ZÉÈÀA-Za-z\-]+)\s+(\d+)')
    subset['nombre'] = subset['nombre'].astype(int)
    return subset[['annais', 'preusuel', 'nombre']]


# 🔧 Préparer les 4 datasets
boys_year = prepare_barchart_data(top_name_overall, 1, 'Top_5_name_year')
girls_year = prepare_barchart_data(top_name_overall, 2, 'Top_5_name_year')
boys_decade = prepare_barchart_data(top_name_overall, 1, 'top_5_name_decade')
girls_decade = prepare_barchart_data(top_name_overall, 2, 'top_5_name_decade')

# 🔧 Curseur d’année partagé
slider = alt.binding_range(
    min=top_name_overall['annais'].min(),
    max=top_name_overall['annais'].max(),
    step=1
)

select_year = alt.selection_point(
    name="Année",
    fields=['annais'],
    bind=slider,
    value=2000
)

# 🔧 Fonction de création de bar chart trié avec labels inclinés


def make_barchart(data, title):
    return alt.Chart(data).mark_bar().encode(
        x=alt.X('preusuel:N', title='Prénom', sort='-y',
                axis=alt.Axis(labelAngle=-30)),
        y=alt.Y('nombre:Q', title='Nombre de naissances'),
        tooltip=['preusuel:N', 'nombre:Q']
    ).transform_filter(
        select_year
    ).properties(
        title=title,
        width=300,
        height=250
    )


# 🔧 Créer les 4 visualisations
chart1 = make_barchart(boys_year, "Garçons - Annuel")
chart2 = make_barchart(girls_year, "Filles - Annuel")
chart3 = make_barchart(boys_decade, "Garçons - Décennal")
chart4 = make_barchart(girls_decade, "Filles - Décennal")

# ✅ Combinaison finale
final_chart = ((chart1 | chart2) & (chart3 | chart4)
               ).add_params(select_year)
final_chart

c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.

alt.VConcatChart(...)

In [6]:
import pandas as pd
import altair as alt

# 🔧 Extraire les 10 années : ici 2000 à 2009
years_range = list(range(2000, 2010))
top10 = top_name_overall[top_name_overall['annais'].isin(years_range)]

# 🔧 Exploser le top 5 annuel
points_df = top10.explode('Top_5_name_year').copy()
points_df[['preusuel', 'nombre']] = points_df['Top_5_name_year'].str.extract(
    r'([A-ZÉÈÀA-Za-z\-]+)\s+(\d+)')
points_df['nombre'] = points_df['nombre'].astype(int)

# ✅ Scatter plot
scatter = alt.Chart(points_df).mark_circle(size=80).encode(
    x=alt.X('annais:O', title="Année"),
    y=alt.Y('nombre:Q', title="Nombre de naissances"),
    color=alt.Color('preusuel:N', title="Prénom"),
    tooltip=['annais:O', 'preusuel:N', 'nombre:Q']
).properties(
    width=600,
    height=400,
    title="Top 5 des prénoms annuels (2000–2009)"
)

scatter

c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [7]:
import pandas as pd
import altair as alt

# 🔧 Préparer le DataFrame
decade_df = top_name_overall[top_name_overall['sexe'] == 1].copy()
decade_df = decade_df.explode('Top_5_name_year').copy()
decade_df[['preusuel', 'nombre']] = decade_df['Top_5_name_year'].str.extract(
    r'([A-ZÉÈÀA-Za-z\-]+)\s+(\d+)')
decade_df['nombre'] = decade_df['nombre'].astype(int)
decade_df['rank'] = decade_df.groupby('annais').cumcount() + 1

# 🔧 Curseur décennal
decade_slider = alt.binding_range(
    min=1900,
    max=2020,
    step=10,
    name="Décennie"
)

select_decade = alt.selection_point(
    fields=['decade_start'],
    bind=decade_slider,
    value=2000,
    name="Sélection"
)

# 🔧 Ajouter la colonne "decade_start"
decade_df['decade_start'] = (decade_df['annais'] // 10) * 10

# ✅ Graph filtré par décennie sélectionnée
chart = alt.Chart(decade_df).mark_text(
    fontSize=12,
    fontWeight='bold',
    align='center'
).encode(
    x=alt.X('annais:O', title='Année'),
    y=alt.Y('rank:O', sort='ascending', title='Rang (1 = top prénom)'),
    text='preusuel:N',
    color=alt.Color('preusuel:N', legend=None)
).transform_filter(
    f"datum.annais >= datum.decade_start && datum.annais < datum.decade_start + 10"
).transform_filter(
    select_decade
).add_params(
    select_decade
).properties(
    width=700,
    height=300,
    title="Top 5 prénoms garçons par rang (filtré par décennie)"
)

chart

c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [8]:
import pandas as pd
import altair as alt

# 🔁 Fusion garçons + filles
both_df = top_name_overall.copy()
both_df = both_df.explode('Top_5_name_year').copy()
both_df[['preusuel', 'nombre']] = both_df['Top_5_name_year'].str.extract(
    r'([A-ZÉÈÀA-Za-z\-]+)\s+(\d+)')
both_df['nombre'] = both_df['nombre'].astype(int)
both_df['rank'] = both_df.groupby(['annais', 'sexe']).cumcount() + 1
both_df['decade_start'] = (both_df['annais'] // 10) * 10
both_df['sexe'] = both_df['sexe'].map({1: 'Garçons', 2: 'Filles'})

# 🎚️ Curseur par décennie
decade_slider = alt.binding_range(min=1900, max=2020, step=10, name="Décennie")
select_decade = alt.selection_point(
    fields=['decade_start'], bind=decade_slider, value=2000, name="Sélection")

# 🔧 Base chart filtrée
base = alt.Chart(both_df).transform_filter(
    f"datum.annais >= datum.decade_start && datum.annais < datum.decade_start + 10"
).transform_filter(select_decade)

# 📈 Lignes
lines = base.mark_line().encode(
    x=alt.X('annais:O', title='Année'),
    y=alt.Y('rank:O', sort='ascending', title='Rang (1 = top prénom)'),
    color=alt.Color('preusuel:N', title='Prénom'),
    detail='preusuel:N'
)

# 🔘 Points
points = base.mark_point(size=60).encode(
    x='annais:O',
    y=alt.Y('rank:O', sort='ascending'),
    color='preusuel:N',
    tooltip=['preusuel:N', 'rank:O', 'annais:O', 'nombre:Q']
)

# 🔤 Labels
labels = base.mark_text(fontSize=12, fontWeight='bold', dy=-10).encode(
    x='annais:O',
    y=alt.Y('rank:O', sort='ascending'),
    text='preusuel:N',
    color=alt.Color('preusuel:N', legend=None)
)

# 🔀 Combinaison des éléments
layer = (lines + points + labels).properties(width=700, height=300)

# ✅ Graphe de rangs avec facet
final_chart = layer.facet(
    row=alt.Row('sexe:N', title=None)
).add_params(select_decade).resolve_scale(color='independent')

# 🔁 Préparer les données pour le bar chart
stack_df = top_name_overall.copy()
stack_df = stack_df.explode('top_5_name_decade').copy()
stack_df[['preusuel', 'nombre']] = stack_df['top_5_name_decade'].str.extract(
    r'([A-ZÉÈÀA-Za-z\-]+)\s+(\d+)')
stack_df['nombre'] = stack_df['nombre'].astype(int)
stack_df['decade_start'] = (stack_df['annais'] // 10) * 10
stack_df['sexe'] = stack_df['sexe'].map({1: 'Garçons', 2: 'Filles'})

# 🔁 Grouper pour éviter les doublons d'années
stack_grouped = stack_df.groupby(
    ['decade_start', 'preusuel', 'sexe'], as_index=False)['nombre'].sum()

# 🔧 Chart de stack bar trié
bar_chart = alt.Chart(stack_grouped).mark_bar().encode(
    x=alt.X('sexe:N', title=None),
    y=alt.Y('nombre:Q', title='Total sur la décennie'),
    color=alt.Color('preusuel:N', title='Prénom'),
    order=alt.Order('nombre:Q', sort='descending'),
    tooltip=['preusuel:N', 'nombre:Q']
).transform_filter(select_decade).properties(
    width=300,
    height=600,
    title="Top 5 prénoms (décennie) – Garçons vs Filles"
)


# ✅ Fusion finale + configuration titre
final_combined = (final_chart | bar_chart).configure_title(
    fontSize=18,
    anchor='start'
).properties(
    title="Top 5 prénoms garçons et filles – évolution + répartition décennale"
)

final_combined

c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.

alt.HConcatChart(...)

In [15]:
# 🔁 Fusion garçons + filles
both_df = top_name_overall.copy()
both_df = both_df.explode('Top_5_name_year').copy()
both_df[['preusuel', 'nombre']] = both_df['Top_5_name_year'].str.extract(
    r'([A-ZÉÈÀA-Za-z\-]+)\s+(\d+)')
both_df['nombre'] = both_df['nombre'].astype(int)
both_df['rank'] = both_df.groupby(['annais', 'sexe']).cumcount() + 1
both_df['decade_start'] = (both_df['annais'] // 10) * 10
both_df['sexe'] = both_df['sexe'].map({1: 'Garçons', 2: 'Filles'})

# 🎚 Curseur par décennie
decade_slider = alt.binding_range(min=1900, max=2020, step=10, name="Décennie")
select_decade = alt.selection_point(
    fields=['decade_start'], bind=decade_slider, value=2000, name="Sélection")

# 🔧 Base chart filtrée
base = alt.Chart(both_df).transform_filter(
    f"datum.annais >= datum.decade_start && datum.annais < datum.decade_start + 10"
).transform_filter(select_decade)

# 📈 Lignes
lines = base.mark_line().encode(
    x=alt.X('annais:O', title='Année'),
    y=alt.Y('rank:O', sort='ascending', title='Rang (1 = top prénom)'),
    color=alt.Color('preusuel:N', title='Prénom'),
    detail='preusuel:N'
)

# 🔘 Points
points = base.mark_point(size=60).encode(
    x='annais:O',
    y=alt.Y('rank:O', sort='ascending'),
    color='preusuel:N',
    tooltip=['preusuel:N', 'rank:O', 'annais:O', 'nombre:Q']
)

# 🎯 Labels uniquement sur la première année de la décennie
labels = base.transform_filter(
    alt.datum.annais == alt.datum.decade_start
).mark_text(
    align='center',
    dy=-15,  # On met le label au-dessus du point (valeur négative)
    fontSize=12,
    fontWeight='bold'
).encode(
    x='annais:O',
    y=alt.Y('rank:O', sort='ascending'),
    text='preusuel:N',
    color=alt.Color('preusuel:N', title='Prénom')  # On garde la légende visible
)

# 🔀 Combinaison des éléments
layer = (lines + points + labels).properties(width=700, height=300)

# ✅ Graphe de rangs avec facet
final_chart = layer.facet(
    row=alt.Row('sexe:N', title=None)
).add_selection(select_decade).resolve_scale(color='independent')

# ✅ Affichage final
final_chart.configure_title(
    fontSize=18,
    anchor='start'
).properties(
    title="Top 5 prénoms garçons et filles – évolution par décennie"
)

c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a 

alt.FacetChart(...)

In [9]:
import pandas as pd

# On part de just_names pour éviter les géométries
df = just_names.copy()

# Agréger par prénom et sexe pour obtenir le total national
grouped = df.groupby(['preusuel', 'sexe'])['nombre'].sum().reset_index()

# Identifier les prénoms apparaissant dans les deux genres
gender_counts = grouped.groupby('preusuel')['sexe'].nunique()
unisex_names = gender_counts[gender_counts == 2].index

# Filtrer les prénoms unisexes
unisex_df = grouped[grouped['preusuel'].isin(unisex_names)]

# Calculer le total pour chaque prénom (tous sexes confondus)
total_counts = unisex_df.groupby('preusuel')['nombre'].sum().reset_index()

# Trier par ordre décroissant
sorted_unisex = total_counts.sort_values(
    by='nombre', ascending=False).reset_index(drop=True)

# Afficher les 20 premiers
print(sorted_unisex.head(20))

     preusuel   nombre
0       MARIE  2256072
1        JEAN  1913130
2      PIERRE   891794
3      MICHEL   818025
4       ANDRÉ   709633
5      JEANNE   556903
6    PHILIPPE   535355
7       LOUIS   523576
8        RENÉ   514560
9       ALAIN   504106
10    JACQUES   480208
11    BERNARD   466906
12     CLAUDE   463020
13     DANIEL   432620
14      ROGER   421888
15       PAUL   420322
16     ROBERT   417053
17  DOMINIQUE   404510
18    GEORGES   402851
19      HENRI   402547


In [10]:
import pandas as pd

# On part de just_names
df = just_names.copy()

# Agrégation par prénom et sexe
grouped = df.groupby(['preusuel', 'sexe'])['nombre'].sum().reset_index()

# Identifier les prénoms présents dans les deux genres
gender_counts = grouped.groupby('preusuel')['sexe'].nunique()
unisex_names = gender_counts[gender_counts == 2].index

# Garder uniquement les prénoms unisexes
unisex_df = grouped[grouped['preusuel'].isin(unisex_names)]

# Pivot pour avoir une colonne par genre
pivot_df = unisex_df.pivot(
    index='preusuel', columns='sexe', values='nombre').fillna(0)

# Renommer les colonnes
pivot_df.columns = ['masculin', 'féminin']

# Ajouter colonne total
pivot_df['total'] = pivot_df['masculin'] + pivot_df['féminin']

# Calcul des proportions
pivot_df['pct_m'] = pivot_df['masculin'] / pivot_df['total']
pivot_df['pct_f'] = pivot_df['féminin'] / pivot_df['total']

# Garder uniquement ceux où chaque genre ≥ 5%
pivot_df = pivot_df[(pivot_df['pct_m'] >= 0.15) & (pivot_df['pct_f'] >= 0.15)]

# Trier par total décroissant
pivot_df = pivot_df.sort_values(by='total', ascending=False).reset_index()

# Afficher les 20 premiers
print(pivot_df[['preusuel', 'masculin', 'féminin', 'total']].head(20))

     preusuel  masculin  féminin   total
0   DOMINIQUE    238623   165887  404510
1     CAMILLE     73761   201738  275499
2        ALIX      4911    21796   26707
3        EDEN     16808     9158   25966
4     CHARLIE     10060    10465   20525
5        DANY     14564     5888   20452
6       SASHA      8380     5800   14180
7       JESSY     10662     1960   12622
8     LOUISON      3146     3987    7133
9        LOÏS      4908     1595    6503
10     ANDREA      3502     1215    4717
11      SWANN      3468      865    4333
12        MAÉ      3373      806    4179
13        MAE      1234     2123    3357
14       YAEL      1615     1666    3281
15     IRENEE      2522      683    3205
16      ADAMA      1868      465    2333
17    JORDANE      1785      356    2141
18     JANICK       774      870    1644
19       JANY       668      900    1568


In [11]:
import pandas as pd
import geopandas as gpd
import altair as alt
import os
import urllib.request

alt.renderers.enable('html')

geojson_url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
geojson_path = "departements.geojson"

if not os.path.exists(geojson_path):
    print("Téléchargement du fichier GeoJSON des départements...")
    urllib.request.urlretrieve(geojson_url, geojson_path)
    print("Téléchargement terminé.")

# Lecture des données
df = pd.read_csv("dpt2020.csv", sep=';')
df = df[df["preusuel"] != "_PRENOMS_RARES"]
df = df[df["annais"].str.isdigit()]
df["annais"] = df["annais"].astype(int)

# Traitement spécifique pour la Corse
corse_rows = df[df["dpt"] == "20"].copy()
corse_rows_2A = corse_rows.copy()
corse_rows_2B = corse_rows.copy()
corse_rows_2A["dpt"] = "2A"
corse_rows_2B["dpt"] = "2B"
df = df[df["dpt"] != "20"]
df = pd.concat([df, corse_rows_2A, corse_rows_2B], ignore_index=True)

# Chargement des départements
depts = gpd.read_file(geojson_path)

# Limite aux années 2000-2020
df = df[(df["annais"] >= 2000) & (df["annais"] <= 2020)]

# Calcul des prénoms dominants
dominants = (
    df.groupby(['dpt', 'annais', 'sexe', 'preusuel'])['nombre']
    .sum()
    .reset_index()
)

# ✅ Garder le prénom dominant par (dpt, sexe, année) même si occurrences faibles
dominants = dominants.sort_values('nombre', ascending=False).drop_duplicates(
    subset=['dpt', 'annais', 'sexe']
)

# Création de toutes les combinaisons possibles
dpt_codes = depts["code"].unique()
years = list(range(2000, 2021))
sexes = [1, 2]
full_index = pd.MultiIndex.from_product(
    [dpt_codes, years, sexes], names=['dpt', 'annais', 'sexe']
)
full_df = pd.DataFrame(index=full_index).reset_index()

# Fusion avec les prénoms dominants connus
dominants['sexe'] = dominants['sexe'].astype(int)
dominants_full = full_df.merge(dominants, on=['dpt', 'annais', 'sexe'], how='left')

# Fusion avec les données géographiques
merged = depts.merge(dominants_full, how='left', left_on='code', right_on='dpt')

# Construction du GeoJSON enrichi
features = []
for _, row in merged.iterrows():
    geometry = row["geometry"].__geo_interface__
    properties = {
        "nom": row["nom"],
        "code": row["code"],
        "preusuel": row["preusuel"] if pd.notnull(row["preusuel"]) else "Inconnu",
        "annais": int(row["annais"]) if pd.notnull(row["annais"]) else None,
        "sexe": str(int(row["sexe"])) if pd.notnull(row["sexe"]) else None,
        "nombre": int(row["nombre"]) if pd.notnull(row["nombre"]) else None,
    }
    features.append({"type": "Feature", "geometry": geometry, "properties": properties})

geojson_data = {"type": "FeatureCollection", "features": features}
geo_data = alt.Data(values=geojson_data["features"])

# Widgets interactifs
year_slider = alt.binding_range(min=2000, max=2020, step=2, name="Année")
select_year = alt.selection_point(
    name="Année",
    fields=["properties.annais"],
    bind=year_slider,
    value=2020
)

gender_menu = alt.binding_radio(options=["1", "2"], name="Sexe", labels=["Garçon", "Fille"])
select_gender = alt.selection_point(
    name="Sexe",
    fields=["properties.sexe"],
    bind=gender_menu,
    value="1"
)

# Carte
base_map = alt.Chart(geo_data).mark_geoshape(
    fill="#eeeeee", stroke="white", strokeWidth=0.5
).encode().properties(width=600, height=600)

map_chart = alt.Chart(geo_data).mark_geoshape().encode(
    color=alt.Color("properties.preusuel:N", title="Prénom dominant"),
    tooltip=[
        alt.Tooltip("properties.nom:N", title="Département"),
        alt.Tooltip("properties.preusuel:N", title="Prénom"),
    ]
).transform_filter(
    select_year
).transform_filter(
    select_gender
).add_params(
    select_year, select_gender
).properties(
    width=600, height=600,
    title="Top prénom par département"
)

(base_map + map_chart).save("carte_prenoms.html")
print("Carte enregistrée dans le fichier 'carte_prenoms.html'. Ouvre-le dans ton navigateur.")

Carte enregistrée dans le fichier 'carte_prenoms.html'. Ouvre-le dans ton navigateur.


In [12]:
import pandas as pd
import altair as alt

# On part de just_names
df = just_names.copy()

# Décennie calculée pour la sélection temporelle
df['decade'] = (df['annais'] // 10) * 10

# Agrégation globale par prénom et sexe
grouped = df.groupby(['preusuel', 'sexe'])['nombre'].sum().reset_index()

# Identifier les prénoms présents dans les deux genres
gender_counts = grouped.groupby('preusuel')['sexe'].nunique()
unisex_names = gender_counts[gender_counts == 2].index

# Garder uniquement les prénoms unisexes
unisex_df = grouped[grouped['preusuel'].isin(unisex_names)]

# Pivot global pour calcul des proportions
pivot_df = unisex_df.pivot(index='preusuel', columns='sexe', values='nombre').fillna(0)
pivot_df.columns = ['masculin', 'féminin']
pivot_df['total'] = pivot_df['masculin'] + pivot_df['féminin']
pivot_df['pct_m'] = pivot_df['masculin'] / pivot_df['total']
pivot_df['pct_f'] = pivot_df['féminin'] / pivot_df['total']

# 🔧 Appliquer ton filtre d’androgynie stricte
pivot_df = pivot_df[(pivot_df['pct_m'] >= 0.15) & (pivot_df['pct_f'] >= 0.15)].reset_index()

# 🔧 Puis sélectionner les 10 prénoms les plus fréquents (après filtre)
pivot_df = pivot_df.sort_values('total', ascending=False).head(10)
filtered_names = pivot_df['preusuel'].tolist()

# Préparer les données annuelles uniquement pour ces prénoms
df_filtered = df[df['preusuel'].isin(filtered_names)]

grouped_year = df_filtered.groupby(['preusuel', 'annais', 'sexe'])['nombre'].sum().reset_index()

# Pivot annuel (année par année)
pivot_year = grouped_year.pivot_table(index=['preusuel', 'annais'], columns='sexe', values='nombre', fill_value=0).reset_index()
pivot_year.columns.name = None
pivot_year = pivot_year.rename(columns={1: 'masculin', 2: 'féminin'})

pivot_year['total'] = pivot_year['masculin'] + pivot_year['féminin']
pivot_year['pct_f'] = pivot_year['féminin'] / pivot_year['total']
pivot_year['balance'] = pivot_year['pct_f'] - 0.5

# Sensibilité de couleur (toujours ajustable ici)
balance_scale = 0.3
pivot_year['balance'] = pivot_year['balance'].clip(-balance_scale, balance_scale)

# Curseur interactif pour filtrer 10 années à la fois
year_slider = alt.binding_range(min=df['annais'].min(), max=df['annais'].max()-9, step=10, name='Période (début) : ')
year_selection = alt.selection_point(fields=['start_year'], bind=year_slider, value=df['annais'].min())

# Créer la colonne start_year pour découpage temporel
pivot_year['start_year'] = (pivot_year['annais'] // 10) * 10

# Base filtrée dynamiquement avec Altair
base = alt.Chart(pivot_year).transform_filter(
    f"datum.annais >= datum.start_year && datum.annais < datum.start_year + 10"
).transform_filter(year_selection)

# Heatmap Altair
heatmap = base.mark_rect().encode(
    x=alt.X('annais:O', title='Année'),
    y=alt.Y('preusuel:N', title='Prénom', sort=filtered_names),
    color=alt.Color(
        'balance:Q',
        scale=alt.Scale(
            domain=[-balance_scale, -0.25, 0, 0.25, balance_scale],
            range=["darkblue", "lightblue", "white", "lightcoral", "darkred"]
        ),
        legend=alt.Legend(title="Féminin ⟶ Masculin")
    ),
    tooltip=['preusuel', 'annais', 'masculin', 'féminin', 'total']
).properties(
    width=700,
    height=400,
    title="Top 10 prénoms androgynes (fenêtre glissante de 10 ans)"
).add_params(year_selection)

alt.data_transformers.disable_max_rows()

heatmap



c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [13]:
import pandas as pd
import altair as alt

# On part de just_names
df = just_names.copy()

# Décennie calculée
df['decade'] = (df['annais'] // 10) * 10

# Agrégation globale pour le filtre androgynie
grouped = df.groupby(['preusuel', 'sexe'])['nombre'].sum().reset_index()

# Identifier prénoms présents dans les deux genres
gender_counts = grouped.groupby('preusuel')['sexe'].nunique()
unisex_names = gender_counts[gender_counts == 2].index

# On filtre sur ces unisexes
unisex_df = grouped[grouped['preusuel'].isin(unisex_names)]

# Pivot global
pivot_df = unisex_df.pivot(index='preusuel', columns='sexe', values='nombre').fillna(0)
pivot_df.columns = ['masculin', 'féminin']
pivot_df['total'] = pivot_df['masculin'] + pivot_df['féminin']
pivot_df['pct_m'] = pivot_df['masculin'] / pivot_df['total']
pivot_df['pct_f'] = pivot_df['féminin'] / pivot_df['total']

# Filtre androgynie
pivot_df = pivot_df[(pivot_df['pct_m'] >= 0.15) & (pivot_df['pct_f'] >= 0.15)].reset_index()
unisex_names_filtered = pivot_df['preusuel'].tolist()

# On travaille uniquement avec les androgynes filtrés
df_filtered = df[df['preusuel'].isin(unisex_names_filtered)]

# Grouper pour connaître le nombre d'années actives par prénom et décennie
count_years = df_filtered.groupby(['preusuel', 'decade'])['annais'].nunique().reset_index(name='year_count')

# On garde ceux qui apparaissent au moins 8 années dans la décennie
count_years = count_years[count_years['year_count'] >= 8]

# 🔧 Correction typage avant merge pour éviter l'erreur
df_filtered['preusuel'] = df_filtered['preusuel'].astype(str)
count_years['preusuel'] = count_years['preusuel'].astype(str)

df_filtered['decade'] = df_filtered['decade'].astype('int64')
count_years['decade'] = count_years['decade'].astype('int64')

# Merge pour ne garder que les lignes valides
df_valid = df_filtered.merge(count_years[['preusuel', 'decade']], on=['preusuel', 'decade'])

# 🔧 Maintenant on sélectionne les 10 prénoms les plus fréquents
total_counts = df_valid.groupby('preusuel')['nombre'].sum().reset_index()
top10_names = total_counts.sort_values(by='nombre', ascending=False).head(10)['preusuel'].tolist()

df_valid = df_valid[df_valid['preusuel'].isin(top10_names)]

# Groupe annuel pour construire la heatmap
grouped_year = df_valid.groupby(['preusuel', 'annais', 'sexe'])['nombre'].sum().reset_index()

# Pivot annuel
pivot_year = grouped_year.pivot_table(index=['preusuel', 'annais'], columns='sexe', values='nombre', fill_value=0).reset_index()
pivot_year.columns.name = None
pivot_year = pivot_year.rename(columns={1: 'masculin', 2: 'féminin'})

pivot_year['total'] = pivot_year['masculin'] + pivot_year['féminin']
pivot_year['pct_f'] = pivot_year['féminin'] / pivot_year['total']
pivot_year['balance'] = pivot_year['pct_f'] - 0.5
balance_scale = 0.3
pivot_year['balance'] = pivot_year['balance'].clip(-balance_scale, balance_scale)

# Curseur temporel
year_slider = alt.binding_range(min=df['annais'].min(), max=df['annais'].max()-9, step=10, name='Période (début) : ')
year_selection = alt.selection_point(fields=['start_year'], bind=year_slider, value=df['annais'].min())

pivot_year['start_year'] = (pivot_year['annais'] // 10) * 10

# Altair Chart
base = alt.Chart(pivot_year).transform_filter(
    f"datum.annais >= datum.start_year && datum.annais < datum.start_year + 10"
).transform_filter(year_selection)

heatmap = base.mark_rect().encode(
    x=alt.X('annais:O', title='Année'),
    y=alt.Y('preusuel:N', title='Prénom', sort=top10_names),
    color=alt.Color(
        'balance:Q',
        scale=alt.Scale(domain=[-balance_scale, -0.25, 0, 0.25, balance_scale],
                        range=["darkblue", "lightblue", "white", "lightcoral", "darkred"]),
        legend=alt.Legend(title="Féminin ⟶ Masculin")
    ),
    tooltip=['preusuel', 'annais', 'masculin', 'féminin', 'total']
).properties(
    width=700,
    height=400,
    title="Top 10 prénoms androgynes actifs par décennie (≥8 ans sur 10)"
).add_params(year_selection)

alt.data_transformers.disable_max_rows()

heatmap

C:\Users\maela\AppData\Local\Temp\ipykernel_23696\3746208591.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['preusuel'] = df_filtered['preusuel'].astype(str)
C:\Users\maela\AppData\Local\Temp\ipykernel_23696\3746208591.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['decade'] = df_filtered['decade'].astype('int64')
c:\Users\maela\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future ve

alt.Chart(...)